# L5: Data Analyst Agent

First start by filtering warnings and loading environment variables.

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [1]:
from utils.helper import load_env
load_env()

Now, let's create the data analyst agent. You will create a sandbox, then open the `pokemon.csv` dataset and write it to the sandbox as `data.csv`.

In [5]:
from utils.utils import create_sandbox

sbx = create_sandbox()

with open("pokemon.csv", "r") as f:
    content = f.read()

sbx.files.write("data.csv", content)

INFO     [sandbox] 🔌 Reconnecting to Sandbox.create(id=iy9vo6icqv9dv7e9avhat)

WriteInfo(name='data.csv', type='file', path='/home/user/data.csv')

Now you can import your `coding_agent` and modify the system prompt to let it know that the user has uploaded a dataset called `data.csv` and it should create interesting plots.

In [6]:
from utils.coding_agent import coding_agent, log
from utils.tools_schemas import execute_code_schema
from utils.tools import execute_code
from openai import OpenAI

client = OpenAI()

system = """You are a senior python programmer. 
You must run the code using the `execute_code` tool.
The user has uploaded a data.csv.
You help the user understanding the data 
by creating interesting plots.
"""

tools = { "execute_code" : execute_code }

Now you can query the data as normal and the sandbox will execute code on the data to answer the queries.

In [7]:
messages = []

query = "What is the data about?"

messages, usage = log(coding_agent,
    messages=messages,
    query=query,
    client=client,
    system=system,
    tools_schemas=[execute_code_schema],
    tools=tools,
    max_steps=10,
    sbx=sbx,
)

╭───────────────────────────────────────────────── 🤖 Tool Call ──────────────────────────────────────────────────╮
│ execute_code                                                                                                    │
│ Arguments: {"code":"import pandas as pd\ndf = pd.read_csv('data.csv')\ndf.head()"}                              │
│ Result: {"results": [{"text": "                     abilities  against_bug  against_dark  against_dragon  \\\n0 │
│ ['Overgrow', 'Chlorophyll']          1.0           1.0             1.0   \n1  ['Overgrow', 'Chl...              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

INFO     ✨: The data appears to be about Pokémon. It contains various attributes for each Pokémon, such as their  
         abilities, effectiveness against different types (e.g., bug, dark, dragon), stats (attack, defense, speed,
         etc.), classification, type, weight, generation, and whether they are legendary. The dataset includes     
         detailed information about each Pokémon, including their name, pokedex number, and other gameplay-related 
         attributes.                                                                                               
                                                                                                                   
         Would you like me to create some plots to visualize specific aspects of this data? For example,           
         distributions of types, stats, legendary vs non-legendary, or something else?

INFO     [agent] 🔢 tokens: 4020 total

In [8]:
query = "Can you aggregate the pokemons by type?"

messages, usage = log(coding_agent,
    messages=messages,
    query=query,
    client=client,
    system=system,
    tools_schemas=[execute_code_schema],
    tools=tools,
    max_steps=10,
    sbx=sbx,
)

╭───────────────────────────────────────────────── 🤖 Tool Call ──────────────────────────────────────────────────╮
│ execute_code                                                                                                    │
│ Arguments: {"code":"# Grouping by primary type and counting the number of Pokemon in each type\ntype_counts =   │
│ df['type1'].value_counts().reset_index()\ntype_counts.columns = ['type', 'count']\ntype_counts"}                │
│ Result: {"results": [{"text": "        type  count\n0      water    114\n1     normal    105\n2      grass      │
│ 78\n3        bug     72\n4    psychic     53\n5       fire     52\n6       rock     45\n7   elect...            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

INFO     ✨: Here is the count of Pokémon aggregated by their primary type:                                        
                                                                                                                   
         | Type      | Count |                                                                                     
         |-----------|--------|                                                                                    
         | water     | 114    |                                                                                    
         | normal    | 105    |                                                                                    
         | grass     | 78     |                                                                                    
         | bug       | 72     |                                                                                    
         | psychic   | 53     |                                                                                    
         | fire      | 52     |                                                                                    
         | rock      | 45     |                                                                                    
         | electric  | 39     |                                                                                    
         | poison    | 32     |                                                                                    
         | ground    | 32     |                                                                                    
         | dark      | 29     |                                                                                    
         | fighting  | 28     |                                                                                    
         | dragon    | 27     |                                                                                    
         | ghost     | 27     |                                                                                    
         | steel     | 24     |                                                                                    
         | ice       | 23     |                                                                                    
         | fairy     | 18     |                                                                                    
         | flying    | 3      |                                                                                    
                                                                                                                   
         Would you like me to visualize this distribution in a plot?

INFO     [agent] 🔢 tokens: 5548 total

### Gradio UI

You can use this provided Gradio UI to have a nicer user experience when chatting with your code generation agent. It will also give interesting information about the context stack, including the number of tokens used in the result, by the assistant, the user, and tools.

In [9]:
from utils.ui import ui

ui(coding_agent,
    messages,
    client=client,
    system=system,
    tools_schemas=[execute_code_schema],
    tools=tools,
    max_steps=10,
    sbx=sbx,
).launch(share=True, height=800)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://b32ffac0cb74878eb7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## Your Turn

Now it's your turn to explore this unknown dataset. Ask questions and learn more about the information inside.

In [10]:
sbx = create_sandbox()
with open("unknown.csv", "rb") as f:
    content = f.read()
sbx.files.write("data.csv", content)

messages = []

ui(
    coding_agent,
    messages,
    client=client,
    system=system,
    tools_schemas=[execute_code_schema],
    tools=tools,
    max_steps=10,
    sbx=sbx,
).launch(share=True, height=800)

INFO     [sandbox] 🔌 Reconnecting to Sandbox.create(id=iy9vo6icqv9dv7e9avhat)

* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://4685c68e13b0ae3fb5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
